# Computing Proximity to the Closest Train Station for all Rental Properties

In [863]:
import requests
import numpy as np
import zipfile
import io
import geopandas as gpd
from openrouteservice import Client
import json
import pandas as pd
from shapely.geometry import Point
import ast
import time
import os
import sys
sys.path.append('../')
from scripts.utils import download_file, extract_zip

## Loading Train Station Data

In [864]:
url = 'https://s3.ap-southeast-2.amazonaws.com/cl-isd-prd-datashare-s3-delivery/Order_FYXPQM.zip'
output_dir = '../data/landing/train_stations'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

# Defining the file directories
zip_path = f'{output_dir}/train_stations.zip'
extract_to = '../data/landing/train_stations/extracted'
    
# Download the ZIP file
download_file(url, zip_path)
    
# Extract the ZIP file
extract_zip(zip_path, extract_to)

# List files in the extracted directory
extracted_files = os.listdir(extract_to)
print("Files in extracted directory:", extracted_files)

Files in extracted directory: ['Creative Commons Licence.html', 'll_gda94', 'PTV_METRO_TRAIN_STATION_b6722101-8db5-51f0-8a6f-d1e4fe805b73.html']


In [865]:
# Path to the shapefile
shapefile_path = '../data/landing/train_stations/extracted/ll_gda94/esrishape/whole_of_dataset/victoria/PTV/PTV_METRO_TRAIN_STATION.shp'

# Read the shapefile into a GeoDataFrame
train_gdf = gpd.read_file(shapefile_path)

# Display the first few rows of the GeoDataFrame
print(train_gdf.head())

# Display the columns of the GeoDataFrame
train_gdf.columns

  STOP_ID   LATITUDE                                          STOP_NAME  \
0   19970 -37.781193             Royal Park Railway Station (Parkville)   
1   19971 -37.788140  Flemington Bridge Railway Station (North Melbo...   
2   19972 -37.794267         Macaulay Railway Station (North Melbourne)   
3   19973 -37.807419   North Melbourne Railway Station (West Melbourne)   
4   19974 -37.788657        Clifton Hill Railway Station (Clifton Hill)   

    LONGITUDE TICKETZONE                                          ROUTEUSSP  \
0  144.952301          1                                            Upfield   
1  144.939323          1                                            Upfield   
2  144.936166          1                                            Upfield   
3  144.942570          1  Flemington,Sunbury,Upfield,Werribee,Williamsto...   
4  144.995417          1                                 Mernda,Hurstbridge   

                      geometry  
0   POINT (144.9523 -37.78119)  
1  POINT

Index(['STOP_ID', 'LATITUDE', 'STOP_NAME', 'LONGITUDE', 'TICKETZONE',
       'ROUTEUSSP', 'geometry'],
      dtype='object')

In [866]:
# Convert column names to lowercase
train_gdf.columns = [col.lower() for col in train_gdf.columns]

# Display the updated GeoDataFrame
train_gdf.head()

,stop_id,latitude,stop_name,longitude,ticketzone,routeussp,geometry
0,19970,-37.781193,Royal Park Railway Station (Parkville),144.952301,1,Upfield,POINT (144.9523 -37.78119)
1,19971,-37.788140,Flemington Bridge Railway Station (North Melbo...,144.939323,1,Upfield,POINT (144.93932 -37.78814)
2,19972,-37.794267,Macaulay Railway Station (North Melbourne),144.936166,1,Upfield,POINT (144.93617 -37.79427)
3,19973,-37.807419,North Melbourne Railway Station (West Melbourne),144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.94257 -37.80742)
4,19974,-37.788657,Clifton Hill Railway Station (Clifton Hill),144.995417,1,"Mernda,Hurstbridge",POINT (144.99542 -37.78866)


In [847]:
# Checking longitude and latitude are within the reasonable limits for Victoria

# Latitude: Approx [-39, -34]
# Longitude: Approx [140, 150]

train_gdf.describe()

,latitude,longitude
count,220.000000,220.000000
mean,-37.852378,145.045691
std,0.140332,0.139902
min,-38.374235,144.661118
25%,-37.899626,144.961156
50%,-37.826147,145.036588
75%,-37.769623,145.121451
max,-37.579091,145.486379


## Loading Preprocessed Domain Rental Data

In [867]:
domain_df = pd.read_parquet('../data/raw/all_domain_properties.parquet')

# Display the DataFrame
domain_df.head()

,name,rooms,parking,property_type,date_available,bond,coordinates,weekly_cost,address,suburb,postcode
0,"6 Gentle Street, Clayton VIC 3168","[3 Beds, 2 Baths]",[2 Parking],House,09/24,$2520,"[-37.9221445, 145.1130263]",580.0,6 gentle street,clayton,3168
1,"142 Civic Parade, Altona VIC 3018","[3 Beds, 1 Bath]",[1 Parking],House,09/24,$2390,"[-37.86445, 144.82913]",550.0,142 civic parade,altona,3018
2,"4/104 Bernard Street, Cheltenham VIC 3192","[3 Beds, 2 Baths]",[2 Parking],Townhouse,09/24,$3302,"[-37.9543583, 145.0693877]",760.0,4/104 bernard street,cheltenham,3192
3,"2 Collendina Crescent, Scoresby VIC 3179","[3 Beds, 2 Baths]",[2 Parking],House,09/24,$2694,"[-37.8901142, 145.223353]",640.0,2 collendina crescent,scoresby,3179
4,"103/6A Evergreen Mews, Armadale VIC 3143","[2 Beds, 2 Baths]",[1 Parking],Apartment / Unit / Flat,09/24,None,"[-37.8518143, 145.0133926]",790.0,103/6a evergreen mews,armadale,3143


In [852]:
# Check the data type of a few entries
print(domain_df['coordinates'].apply(type).value_counts())

coordinates
<class 'numpy.ndarray'>    13017
<class 'NoneType'>            44
Name: count, dtype: int64


In [868]:
def extract_latitude(coords):
    if isinstance(coords, (list, tuple, np.ndarray)) and len(coords) > 0:
        return float(coords[0])
    return None

def extract_longitude(coords):
    if isinstance(coords, (list, tuple, np.ndarray)) and len(coords) > 1:
        return float(coords[1])
    return None

# Apply the functions to handle both numpy.ndarray and NoneType
domain_df['latitude'] = domain_df['coordinates'].apply(extract_latitude)
domain_df['longitude'] = domain_df['coordinates'].apply(extract_longitude)

domain_df.head()

,name,rooms,parking,property_type,date_available,bond,coordinates,weekly_cost,address,suburb,postcode,latitude,longitude
0,"6 Gentle Street, Clayton VIC 3168","[3 Beds, 2 Baths]",[2 Parking],House,09/24,$2520,"[-37.9221445, 145.1130263]",580.0,6 gentle street,clayton,3168,-37.922145,145.113026
1,"142 Civic Parade, Altona VIC 3018","[3 Beds, 1 Bath]",[1 Parking],House,09/24,$2390,"[-37.86445, 144.82913]",550.0,142 civic parade,altona,3018,-37.864450,144.829130
2,"4/104 Bernard Street, Cheltenham VIC 3192","[3 Beds, 2 Baths]",[2 Parking],Townhouse,09/24,$3302,"[-37.9543583, 145.0693877]",760.0,4/104 bernard street,cheltenham,3192,-37.954358,145.069388
3,"2 Collendina Crescent, Scoresby VIC 3179","[3 Beds, 2 Baths]",[2 Parking],House,09/24,$2694,"[-37.8901142, 145.223353]",640.0,2 collendina crescent,scoresby,3179,-37.890114,145.223353
4,"103/6A Evergreen Mews, Armadale VIC 3143","[2 Beds, 2 Baths]",[1 Parking],Apartment / Unit / Flat,09/24,None,"[-37.8518143, 145.0133926]",790.0,103/6a evergreen mews,armadale,3143,-37.851814,145.013393


In [869]:
# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(domain_df['longitude'], domain_df['latitude'])]
domain_gdf = gpd.GeoDataFrame(domain_df, geometry=geometry)

# Set the Coordinate Reference System (CRS) if known, e.g., EPSG:4326 for WGS84
domain_gdf.set_crs(epsg=4326, inplace=True)

# Display the first few rows of the GeoDataFrame
domain_gdf.head()

,name,rooms,parking,property_type,date_available,bond,coordinates,weekly_cost,address,suburb,postcode,latitude,longitude,geometry
0,"6 Gentle Street, Clayton VIC 3168","[3 Beds, 2 Baths]",[2 Parking],House,09/24,$2520,"[-37.9221445, 145.1130263]",580.0,6 gentle street,clayton,3168,-37.922145,145.113026,POINT (145.11303 -37.92214)
1,"142 Civic Parade, Altona VIC 3018","[3 Beds, 1 Bath]",[1 Parking],House,09/24,$2390,"[-37.86445, 144.82913]",550.0,142 civic parade,altona,3018,-37.864450,144.829130,POINT (144.82913 -37.86445)
2,"4/104 Bernard Street, Cheltenham VIC 3192","[3 Beds, 2 Baths]",[2 Parking],Townhouse,09/24,$3302,"[-37.9543583, 145.0693877]",760.0,4/104 bernard street,cheltenham,3192,-37.954358,145.069388,POINT (145.06939 -37.95436)
3,"2 Collendina Crescent, Scoresby VIC 3179","[3 Beds, 2 Baths]",[2 Parking],House,09/24,$2694,"[-37.8901142, 145.223353]",640.0,2 collendina crescent,scoresby,3179,-37.890114,145.223353,POINT (145.22335 -37.89011)
4,"103/6A Evergreen Mews, Armadale VIC 3143","[2 Beds, 2 Baths]",[1 Parking],Apartment / Unit / Flat,09/24,None,"[-37.8518143, 145.0133926]",790.0,103/6a evergreen mews,armadale,3143,-37.851814,145.013393,POINT (145.01339 -37.85181)


## Calculating distances to closest train station

In [870]:
# Make sure CRS is EPSG 4326 for Rental Data
domain_gdf = domain_gdf.to_crs(epsg=4326)
print(domain_gdf.crs)

EPSG:4326


In [871]:
# Make sure CRS is EPSG 4326 for Train Data
train_gdf = train_gdf.to_crs(epsg=4326)
print(train_gdf.crs)

EPSG:4326


In [872]:
# Initialize OpenRouteService client with your API key
api_key = '5b3ce3597851110001cf62483425e548d4ed4c2ca4a6b5814be96094'
client = Client(key=api_key)

## Creating Train Station Isochrones

In [702]:
# THIS TAKES A WHILE. EXPECT IT TO BE ABOUT 13 MINUTES

def generate_isochrones(train_gdf, client, processed_stations, output_file, interval=600):  # 600 seconds = 10 minutes
    isochrone_features = []  # To store all isochrone features
    
    for index, train_station in train_gdf.iterrows():
        station_id = train_station['stop_id']  # Use STOP_ID
        station_name = train_station['stop_name']  # Use STOP_NAME
        lat, lon = train_station.geometry.y, train_station.geometry.x
        
        # Skip if station already processed
        if station_id in processed_stations:
            continue
        
        print(f"Processing: {station_id} - {station_name} (Lat: {lat}, Lon: {lon})")
        
        # Isochrone parameters for 10 minutes (600 seconds) driving time
        params_iso = {
            'profile': 'driving-car',
            'intervals': [interval],  # Single interval of 10 minutes
            'locations': [[lon, lat]],
        }
        
        try:
            # Fetch isochrones
            isochrones = client.isochrones(**params_iso)
            
            # Convert the isochrone to a GeoJSON Feature
            for feature in isochrones['features']:
                isochrone_feature = {
                    'type': 'Feature',
                    'geometry': feature['geometry'],  # Isochrone geometry
                    'properties': {
                        'station_id': station_id,  # Use STOP_ID
                        'station_name': station_name,  # Use STOP_NAME
                        'interval': interval,  # 10-minute interval
                        'latitude': lat,
                        'longitude': lon
                    }
                }
                isochrone_features.append(isochrone_feature)
            
            # Mark station as processed
            processed_stations.append(station_id)

            # Delay to prevent hitting the rate limit
            delay = 3
            time.sleep(delay)
            
        except Exception as e:
            print(f"Error processing station {station_id} - {station_name}: {e}")
    
    # Print the count and sample of isochrone features
    print(f"Isochrone features count: {len(isochrone_features)}")
    print("Sample isochrone features:", json.dumps(isochrone_features[:1], indent=2))
    
    # Create a GeoJSON structure to hold all isochrones
    isochrone_collection = {
        'type': 'FeatureCollection',
        'features': isochrone_features
    }
    
    # Ensure the directory exists
    output_directory = os.path.dirname(output_file)
    os.makedirs(output_directory, exist_ok=True)
    
    # Save the entire collection to a single GeoJSON file
    with open(output_file, 'w') as f:
        json.dump(isochrone_collection, f, indent=2)
    
    print(f"All isochrones saved to {output_file}")

# Prepare test parameters
processed_stations = []

# Run the test
generate_isochrones(
    train_gdf, 
    client, 
    processed_stations, 
    '../data/curated/isochrones/isochrones.geojson',
    interval=600
)


Processing: 19970 - Royal Park Railway Station (Parkville) (Lat: -37.7811929725527, Lon: 144.95230120580877)
Processing: 19971 - Flemington Bridge Railway Station (North Melbourne) (Lat: -37.788139984393844, Lon: 144.93932321237276)
Processing: 19972 - Macaulay Railway Station (North Melbourne) (Lat: -37.79426700459872, Lon: 144.93616600406753)
Processing: 19973 - North Melbourne Railway Station (West Melbourne) (Lat: -37.80741897361899, Lon: 144.94257002890663)
Processing: 19974 - Clifton Hill Railway Station (Clifton Hill) (Lat: -37.78865703363607, Lon: 144.9954169601631)
Processing: 19975 - Victoria Park Railway Station (Abbotsford) (Lat: -37.79915796626723, Lon: 144.9944510689111)
Processing: 19976 - Collingwood Railway Station (Abbotsford) (Lat: -37.80452601640281, Lon: 144.99375014165992)
Processing: 19977 - North Richmond Railway Station (Richmond) (Lat: -37.8103979761548, Lon: 144.9924998535909)
Processing: 19978 - West Richmond Railway Station (Richmond) (Lat: -37.814949005848

## Finding which Train Station Isochrones each Rental Property Falls In

In [873]:
isochrone_gdf = gpd.read_file('../data/curated/isochrones/isochrones.geojson')

# Display the first few rows to verify
isochrone_gdf.head()

,station_id,station_name,interval,latitude,longitude,geometry
0,19970,Royal Park Railway Station (Parkville),600,-37.781193,144.952301,"POLYGON ((144.89721 -37.76938, 144.89832 -37.7..."
1,19971,Flemington Bridge Railway Station (North Melbo...,600,-37.788140,144.939323,"POLYGON ((144.8873 -37.77075, 144.8921 -37.774..."
2,19972,Macaulay Railway Station (North Melbourne),600,-37.794267,144.936166,"POLYGON ((144.89347 -37.76897, 144.8941 -37.77..."
3,19973,North Melbourne Railway Station (West Melbourne),600,-37.807419,144.942570,"POLYGON ((144.90048 -37.79937, 144.89971 -37.8..."
4,19974,Clifton Hill Railway Station (Clifton Hill),600,-37.788657,144.995417,"POLYGON ((144.95551 -37.78951, 144.95568 -37.7..."


In [874]:
# Perform Spatial join to find which isochrones each Rental Property falls in
joined_gdf = gpd.sjoin(domain_gdf, isochrone_gdf, how='inner', predicate='within')

In [875]:
joined_gdf

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,latitude_left,longitude_left,geometry,index_right,station_id,station_name,interval,latitude_right,longitude_right
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),71,52095,Southland Railway Station (Cheltenham),600,-37.958756,145.049121
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),114,19864,Parkdale Railway Station (Parkdale),600,-37.993079,145.076327
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),115,19865,Mentone Railway Station (Mentone),600,-37.981865,145.065166
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),116,19866,Cheltenham Railway Station (Cheltenham),600,-37.966650,145.054558
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),122,19872,Highett Railway Station (Highett),600,-37.948425,145.041872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.domain.com.au/3-lauren-square-pakenham-vic-3810-17141332,"3 Lauren Square, Pakenham VIC 3810",$500 per week,"[3 Beds, 1 Bath]",[2 Parking],APPLY NOW: We invite prospective renters to ap...,House,Available Now,$2173,[],"[-38.0730438, 145.4890878]",-38.073044,145.489088,POINT (145.48909 -38.07304),130,19880,Pakenham Railway Station (Pakenham),600,-38.080614,145.486379
https://www.domain.com.au/1-7-carluke-close-berwick-vic-3806-17040706,"1/7 Carluke Close, Berwick VIC 3806",$375/wk,"[0 Beds, 1 Bath]",[1 Parking],* Unverified feature,Studio,Available Now,$1629,"[Heating*, Ensuite]","[-38.0505947, 145.3413087]",-38.050595,145.341309,POINT (145.34131 -38.05059),131,19881,Officer Railway Station (Officer),600,-38.066146,145.410987
https://www.domain.com.au/1-7-carluke-close-berwick-vic-3806-17040706,"1/7 Carluke Close, Berwick VIC 3806",$375/wk,"[0 Beds, 1 Bath]",[1 Parking],* Unverified feature,Studio,Available Now,$1629,"[Heating*, Ensuite]","[-38.0505947, 145.3413087]",-38.050595,145.341309,POINT (145.34131 -38.05059),132,19882,Beaconsfield Railway Station (Beaconsfield),600,-38.050831,145.366074
https://www.domain.com.au/1-7-carluke-close-berwick-vic-3806-17040706,"1/7 Carluke Close, Berwick VIC 3806",$375/wk,"[0 Beds, 1 Bath]",[1 Parking],* Unverified feature,Studio,Available Now,$1629,"[Heating*, Ensuite]","[-38.0505947, 145.3413087]",-38.050595,145.341309,POINT (145.34131 -38.05059),133,19883,Berwick Railway Station (Berwick),600,-38.040408,14

## Calculating Distance to nearby Train Stations for each Rental Property

In [876]:
# Function to get distance using ORS API
def get_distance(lat1, lon1, lat2, lon2):
    # Coordinates for rental property and train station
    coordinates = [[lon1, lat1], [lon2, lat2]]
    
    try:
        # Request route
        route = client.directions(coordinates=coordinates, profile='driving-car', format='geojson')
        # Get distance in meters
        distance = route['features'][0]['properties']['segments'][0]['distance']
        return distance / 1000  # Convert to kilometers
    except Exception as e:
        print(f"Error occurred: {e}")
        return None  # Handle the case where API call fails

# Apply the function to DataFrame with a delay
def apply_distance_with_delay(row):
    distance = get_distance(row['latitude_left'], row['longitude_left'], row['latitude_right'], row['longitude_right'])
    time.sleep(2)  # Adding a delay of 1 second between requests
    return distance

# Apply with delay
joined_gdf['distance_km'] = joined_gdf.apply(apply_distance_with_delay, axis=1)


Error occurred: HTTP Error: 502
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})
Error occurred: 403 ({'error': 'Quota exceeded'})


KeyboardInterrupt: 

In [881]:
joined_gdf.head()

,name,cost_text,rooms,parking,desc,property_type,date_available,bond,property_features,coordinates,latitude_left,longitude_left,geometry,index_right,station_id,station_name,interval,latitude_right,longitude_right
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),71,52095,Southland Railway Station (Cheltenham),600,-37.958756,145.049121
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),114,19864,Parkdale Railway Station (Parkdale),600,-37.993079,145.076327
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),115,19865,Mentone Railway Station (Mentone),600,-37.981865,145.065166
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),116,19866,Cheltenham Railway Station (Cheltenham),600,-37.966650,145.054558
https://www.domain.com.au/72a-argus-street-cheltenham-vic-3192-17104065,"72A Argus Street, Cheltenham VIC 3192","$1,050 Per Week","[4 Beds, 3 Baths]",[2 Parking],Brand new and with a commanding street presenc...,Townhouse,Available Now,$6300,"[Split System Air Con, Split System Heating, O...","[-37.9592116, 145.0649156]",-37.959212,145.064916,POINT (145.06492 -37.95921),122,19872,Highett Railway Station (Highett),600,-37.948425,145.041872


- now finding the closest train station based on minimum distance

In [878]:
# Group by 'name' (property name), and find the row with the minimum 'distance_km' for each group
joined_gdf_min_distance = joined_gdf.sort_values('distance_km').drop_duplicates(subset=['name'], keep='first')

# Reset index to clean up the DataFrame
joined_gdf_min_distance = joined_gdf_min_distance.reset_index(drop=True)

# Remove unecessary columns
joined_gdf_min_distance = joined_gdf_min_distance.drop(columns=['interval', 'index_right', 'latitude_right', 'longitude_right'])

# Display the updated DataFrame
joined_gdf_min_distance


KeyError: 'distance_km'